In [2]:
import pandas as pd
import yfinance as yf
from load_data import get_data_csv
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

csv_files = ['overview.csv','balance.csv','cashflow.csv','income.csv','earnings.csv','data_an.csv','data_qu.csv']

dataframes = get_data_csv(csv_files)

for key, dataframe in dataframes.items():
    globals()[key] = dataframe


In [3]:
overview

,Symbol,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,Industry,Address,FiscalYearEnd,LatestQuarter,MarketCapitalization,EBITDA,PERatio,PEGRatio,BookValue,DividendPerShare,DividendYield,EPS,RevenuePerShareTTM,ProfitMargin,OperatingMarginTTM,ReturnOnAssetsTTM,ReturnOnEquityTTM,RevenueTTM,GrossProfitTTM,DilutedEPSTTM,QuarterlyEarningsGrowthYOY,QuarterlyRevenueGrowthYOY,AnalystTargetPrice,AnalystRatingStrongBuy,AnalystRatingBuy,AnalystRatingHold,AnalystRatingSell,AnalystRatingStrongSell,TrailingPE,ForwardPE,PriceToSalesRatioTTM,PriceToBookRatio,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
0,AAPL,Common Stock,Apple Inc,Apple Inc. is an American multinational techno...,320193,NASDAQ,USD,USA,TECHNOLOGY,ELECTRONIC COMPUTERS,"ONE INFINITE LOOP, CUPERTINO, CA, US",September,2024-03-31,2874070270000,129629004000,29.15,2.223,4.837,0.96,0.0053,6.43,24.54,0.263,0.307,0.221,1.472,381623009000,170782000000,6.43,0.007,-0.043,202.36,10,17,12,2,0,29.15,28.57,7.53,38.74,7.63,21.89,1.264,199.1,163.86,172.89,180.85,15334100000,2024-05-16,2024-05-10


In [5]:
fundamentals


NameError: name 'fundamentals' is not defined

In [94]:
def convert_to_numeric(df,exception_columns):
    for column in df.columns:
        if column not in exception_columns:
            df[column] = pd.to_numeric(df[column], errors='coerce') / 1000000
    return df

In [95]:
exception_columns = ['fiscalDateEnding', 'reportedCurrency','ReportType']

income = convert_to_numeric(income,exception_columns)
balance = convert_to_numeric(balance,exception_columns)
cashflow = convert_to_numeric(cashflow,exception_columns)
earnings['reportedEPS'] = pd.to_numeric(earnings['reportedEPS'])

In [96]:
def process_data(cashflow,income,balance,earnings):
    #Erstellen von Dataframes zur weiteren Verarbeitung
    def select_data(df, report_type, keep=None):
        selected_df = df[df['ReportType'] == report_type].copy()
        selected_df.drop_duplicates(subset=['fiscalDateEnding'], keep='first', inplace=True)
        if keep is None:
            return selected_df.reset_index(drop=True)
        else:
            cols_to_drop = [col for col in selected_df.columns if col not in keep]
            selected_df.drop(columns=cols_to_drop, inplace=True)
            
        return selected_df.reset_index(drop=True)

    def finalise_dataframe(dfs, min_rows=None):
        if min_rows is not None:
            dfs_trimmed = [df.iloc[:min_rows] for df in dfs]
        else:
            dfs_trimmed = dfs
            
        concatenated_df = pd.concat(dfs_trimmed, axis=1)        
        concatenated_df.drop(columns=['ReportType'], inplace=True)           
        concatenated_df = concatenated_df.loc[:, ~concatenated_df.columns.duplicated()]
        concatenated_df = concatenated_df.fillna(0)
        
        return concatenated_df
      
 
    cashflow_qu = select_data(cashflow,'Quarterly')
    balance_qu = select_data(balance,'Quarterly')
    income_qu = select_data(income,'Quarterly')
    earnings_qu = select_data(earnings,'Quarterly',keep=['fiscalDateEnding','reportedEPS'])

    cashflow_an = select_data(cashflow,'Annual')
    balance_an = select_data(balance,'Annual')
    income_an = select_data(income,'Annual')
    earnings_an = select_data(earnings,'Annual',keep=['fiscalDateEnding','reportedEPS'])
    
    income_an = pd.merge(income_an, earnings_an, how='inner',on='fiscalDateEnding')
    income_qu = pd.merge(income_qu, earnings_qu, how='inner',on='fiscalDateEnding')

    dfs_an = [cashflow_an, income_an, balance_an]
    dfs_qu = [cashflow_qu, income_qu]
    
    min_rows_an = min(len(df) for df in dfs_an)
    min_rows_qu = min(len(df) for df in dfs_qu)
    
    all_reports_an = finalise_dataframe(dfs_an, min_rows = min_rows_an)
    all_reports_qu = finalise_dataframe(dfs_qu, min_rows = min_rows_qu)
    
    return all_reports_an, all_reports_qu

In [97]:
all_reports_an, all_reports_qu = process_data(cashflow,income,balance,earnings)
all_reports_an

,fiscalDateEnding,reportedCurrency,operatingCashflow,paymentsForOperatingActivities,proceedsFromOperatingActivities,changeInOperatingLiabilities,changeInOperatingAssets,depreciationDepletionAndAmortization,capitalExpenditures,changeInReceivables,changeInInventory,profitLoss,cashflowFromInvestment,cashflowFromFinancing,proceedsFromRepaymentsOfShortTermDebt,paymentsForRepurchaseOfCommonStock,paymentsForRepurchaseOfEquity,paymentsForRepurchaseOfPreferredStock,dividendPayout,dividendPayoutCommonStock,dividendPayoutPreferredStock,proceedsFromIssuanceOfCommonStock,proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet,proceedsFromIssuanceOfPreferredStock,proceedsFromRepurchaseOfEquity,proceedsFromSaleOfTreasuryStock,changeInCashAndCashEquivalents,changeInExchangeRate,netIncome,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,investmentIncomeNet,netInterestIncome,interestIncome,interestExpense,nonInterestIncome,otherNonOperatingIncome,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,reportedEPS,totalAssets,totalCurrentAssets,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,inventory,currentNetReceivables,totalNonCurrentAssets,propertyPlantEquipment,accumulatedDepreciationAmortizationPPE,intangibleAssets,intangibleAssetsExcludingGoodwill,goodwill,investments,longTermInvestments,shortTermInvestments,otherCurrentAssets,otherNonCurrentAssets,totalLiabilities,totalCurrentLiabilities,currentAccountsPayable,deferredRevenue,currentDebt,shortTermDebt,totalNonCurrentLiabilities,capitalLeaseObligations,longTermDebt,currentLongTermDebt,longTermDebtNoncurrent,shortLongTermDebtTotal,otherCurrentLiabilities,otherNonCurrentLiabilities,totalShareholderEquity,treasuryStock,retainedEarnings,commonStock,commonStockSharesOutstanding
0,2023-09-30,USD,110543.0,5703.0,0.0,1142.0,7719.0,11519.0,10959.0,417.0,1618.0,96995.0,3705.0,-108488.0,-7956.0,77550.0,77550.0,0.0,15025.0,15025.0,0.0,0.0,5228.0,0.0,-77550.0,0.0,5760.0,0.0,96995.0,169148.0,379352.0,239069.0,214137.0,114301.0,24932.0,29915.0,54847.0,3750.0,-3933.0,183.0,3933.0,383285.0,-382.0,8500.0,11519.0,113736.0,16741.0,3933.0,96995.0,96652.0,117669.0,125820.0,6.1200,352583.0,143566.0,29965.0,61555.0,6331.0,60985.0,209017.0,43715.0,70884.0,0.0,0.0,0.0,264965.0,100544.0,31590.0,14695.0,64758.0,290437.0,145308.0,62611.0,20161.0,15972.0,5985.0,145129.0,859.0,105100.0,9822.0,95281.0,111088.0,58829.0,49848.0,62146.0,0.0,-214.0,73812.0,15550.06100
1,2022-09-30,USD,122151.0,4665.0,0.0,15558.0,14358.0,11104.0,10708.0,9343.0,-1484.0,99803.0,-22354.0,-110749.0,7910.0,89402.0,89402.0,0.0,14841.0,14841.0,0.0,0.0,5465.0,0.0,-89402.0,0.0,-10952.0,0.0,99803.0,170782.0,391397.0,248640.0,223546.0,119437.0,25094.0,26251.0,51345.0,2825.0,-2931.0,106.0,2931.0,394328.0,-228.0,8700.0,11104.0,119103.0,19300.0,2931.0,99803.0,88531.0,122034.0,130541.0,6.1100,352755.0,135405.0,23646.0,48304.0,4946.0,60932.0,217350.0,42117.0,72340.0,0.0,0.0,0.0,292870.0,120805.0,24658.0,21223.0,54428.0,302083.0,153982.0,64115.0,20312.0,21239.0,9982.0,148101.0,812.0,110087.0,11128.0,98959.0,233256.0,60845.0,49142.0,50672.0,0.0,-3068.0,64849.0,15943.42500
2,2021-09-30,USD,104038.0,4087.0,0.0,19801.0,24712.0,11284.0,11085.0,14028.0,2642.0,94680.0,-14545.0,-93353.0,2044.0,85971.0,85971.0,0.0,14467.0,14467.0,0.0,1105.0,20393.0,0.0,-84866.0,0.0,-3860.0,0.0,94680.0,152836.0,363172.0,234954.0,212981.0,108949.0,21973.0,21914.0,43887.0,2843.0,-2645.0,2843.0,2645.0,365817.0,60.0,9500.0,11284.0,109207.0,14527.0,2645.0,94680.0,95249.0,111852.0,120233.0,5.6200,351002.0,134836.0,34940.0,62639.0,6580.0,51506.0,216166.0,39440.0,70283.0,0.0,0.0,0.0,318994.0,127877.0,27699.0,14111.0,48849.0,287912.0,125481.0,54763.0,19512.0,15692.0,6000.0,162431.0,769.0,118700.0,9613.0,109106.0,241783.0,47493.0,53325.0,63090.0,0.0,5562.0,

In [87]:
all_reports_qu.iloc[:4]

,fiscalDateEnding,reportedCurrency,operatingCashflow,paymentsForOperatingActivities,proceedsFromOperatingActivities,changeInOperatingLiabilities,changeInOperatingAssets,depreciationDepletionAndAmortization,capitalExpenditures,changeInReceivables,changeInInventory,profitLoss,cashflowFromInvestment,cashflowFromFinancing,proceedsFromRepaymentsOfShortTermDebt,paymentsForRepurchaseOfCommonStock,paymentsForRepurchaseOfEquity,paymentsForRepurchaseOfPreferredStock,dividendPayout,dividendPayoutCommonStock,dividendPayoutPreferredStock,proceedsFromIssuanceOfCommonStock,proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet,proceedsFromIssuanceOfPreferredStock,proceedsFromRepurchaseOfEquity,proceedsFromSaleOfTreasuryStock,changeInCashAndCashEquivalents,changeInExchangeRate,netIncome,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,investmentIncomeNet,netInterestIncome,interestIncome,interestExpense,nonInterestIncome,otherNonOperatingIncome,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,reportedEPS
0,2024-03-31,USD,22690.0,0.0,0.0,-11740.0,-5976.0,2836.0,1996.0,-8767.0,-190.0,23636.0,-310.0,-30433.0,4.0,23205.0,23205.0,0.0,3710.0,3710.0,0.0,0.0,0.0,0.0,-23205.0,0.0,0.0,0.0,23636.0,42271.0,90753.0,54950.0,48482.0,27900.0,6468.0,7903.0,14371.0,0.0,0.0,0.0,0.0,90753.0,0.0,0.0,2836.0,28058.0,4422.0,0.0,23636.0,24054.0,27900.0,30736.0,1.53
1,2023-12-31,USD,39895.0,0.0,0.0,-8407.0,-9530.0,2848.0,2392.0,-11124.0,137.0,33916.0,1927.0,-30585.0,-7968.0,20139.0,20139.0,0.0,3825.0,3825.0,0.0,0.0,0.0,0.0,-20139.0,0.0,0.0,0.0,33916.0,54855.0,119575.0,71506.0,64720.0,40373.0,6786.0,7696.0,14482.0,0.0,0.0,0.0,1002.0,119575.0,0.0,0.0,2848.0,40323.0,6407.0,0.0,33916.0,35990.0,40373.0,43221.0,2.18
2,2023-09-30,USD,21598.0,3113.0,0.0,14946.0,21006.0,2653.0,2163.0,21137.0,-952.0,22956.0,2394.0,-23153.0,3986.0,21003.0,21003.0,0.0,3758.0,3758.0,0.0,0.0,0.0,0.0,-21003.0,0.0,839.0,0.0,22956.0,40427.0,88496.0,55222.0,49071.0,26969.0,6151.0,7307.0,13458.0,984.0,-1002.0,18.0,1002.0,89498.0,47.0,0.0,2653.0,26998.0,4042.0,1002.0,22956.0,23305.0,28000.0,29622.0,1.46
3,2023-06-30,USD,26380.0,0.0,0.0,5203.0,4454.0,3052.0,2093.0,3661.0,22.0,19881.0,437.0,-24048.0,0.0,17478.0,0.0,0.0,3849.0,3849.0,0.0,0.0,0.0,0.0,0.0,0.0,2769.0,0.0,19881.0,36413.0,80799.0,51357.0,45384.0,22998.0,5973.0,7442.0,13415.0,980.0,-998.0,18.0,998.0,81797.0,-247.0,0.0,3052.0,22733.0,2852.0,998.0,19881.0,19826.0,23731.0,26050.0,1.26


In [98]:
def caculate_avg_price_by_year(all_reports_an):
    #Laden der Daten aus yF
    min_date_an = all_reports_an['fiscalDateEnding'].min()
    max_date = pd.Timestamp.today().strftime('%Y-%m-%d')
    data_an = yf.download(ticker,start=min_date_an, end=max_date).reset_index()
    data_an['year'] = data_an['Date'].dt.year
    
    #Berechnung avg_price
    all_reports_an['fiscalDateEnding'] = pd.to_datetime(all_reports_an['fiscalDateEnding'])
    all_reports_an['year'] = all_reports_an['fiscalDateEnding'].dt.year
    average_close_by_year = data_an.groupby('year')['Close'].mean().reset_index()
    average_close_by_year.columns = ['year', 'avg_Kurs']
    average_close_by_year['avg_Kurs'] = round(average_close_by_year['avg_Kurs'].astype(float),2)

    # Merge Price mit Dataframe
    all_reports_an = pd.merge(all_reports_an, average_close_by_year, how='inner',on='year')
    all_reports_an.drop(columns=['fiscalDateEnding','reportedCurrency'], inplace=True)
    return all_reports_an

def create_ttm_dataframe(all_reports_qu,balance_qu):
    #Laden der Daten aus yF
    all_reports_qu = all_reports_qu.iloc[:4]
    min_date_qu = all_reports_qu['fiscalDateEnding'].min()
    data_qu = yf.download(ticker,start=min_date_qu, end=max_date)
    
    #Berechnung avg_price
    average_close_by_ttm = data_qu.Close.mean()
    ttm = all_reports_qu.select_dtypes(include=['number']).sum(axis=0)
    ttm = pd.DataFrame(ttm).transpose()
    ttm['year'] ='ttm'
    ttm['avg_Kurs'] = average_close_by_ttm
    # Merge Price mit Dataframe
    balance_ttm = balance_qu[balance_qu['fiscalDateEnding'] == balance_qu['fiscalDateEnding'].max()].drop(columns = ['ReportType','fiscalDateEnding','reportedCurrency'])
    ttm = pd.concat([ttm,balance_ttm], axis =1)
    return ttm


In [99]:
all_reports_test = caculate_avg_price_by_year(all_reports_an)
all_reports_test

Failed to get ticker 'AAPL' reason: HTTPSConnectionPool(host='fc.yahoo.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)')))
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['AAPL']: Exception('%ticker%: No timezone found, symbol may be delisted')


AttributeError: 'Index' object has no attribute 'year'

In [102]:
cashflow

,fiscalDateEnding,reportedCurrency,operatingCashflow,paymentsForOperatingActivities,proceedsFromOperatingActivities,changeInOperatingLiabilities,changeInOperatingAssets,depreciationDepletionAndAmortization,capitalExpenditures,changeInReceivables,changeInInventory,profitLoss,cashflowFromInvestment,cashflowFromFinancing,proceedsFromRepaymentsOfShortTermDebt,paymentsForRepurchaseOfCommonStock,paymentsForRepurchaseOfEquity,paymentsForRepurchaseOfPreferredStock,dividendPayout,dividendPayoutCommonStock,dividendPayoutPreferredStock,proceedsFromIssuanceOfCommonStock,proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet,proceedsFromIssuanceOfPreferredStock,proceedsFromRepurchaseOfEquity,proceedsFromSaleOfTreasuryStock,changeInCashAndCashEquivalents,changeInExchangeRate,netIncome,ReportType
0,2023-09-30,USD,110543.0,5703.0,NaN,1142.0,7719.0,11519.0,10959.0,417.0,1618.0,96995.0,3705.0,-108488.0,-7956.0,77550.0,77550.0,NaN,15025.0,15025.0,NaN,NaN,5228.0,NaN,-77550.0,NaN,5760.0,NaN,96995.0,Annual
1,2022-09-30,USD,122151.0,4665.0,NaN,15558.0,14358.0,11104.0,10708.0,9343.0,-1484.0,99803.0,-22354.0,-110749.0,7910.0,89402.0,89402.0,NaN,14841.0,14841.0,NaN,NaN,5465.0,NaN,-89402.0,NaN,-10952.0,NaN,99803.0,Annual
2,2021-09-30,USD,104038.0,4087.0,NaN,19801.0,24712.0,11284.0,11085.0,14028.0,2642.0,94680.0,-14545.0,-93353.0,2044.0,85971.0,85971.0,NaN,14467.0,14467.0,NaN,1105.0,20393.0,NaN,-84866.0,NaN,-3860.0,NaN,94680.0,Annual
3,2020-09-30,USD,80674.0,4502.0,NaN,6935.0,1245.0,11056.0,7309.0,-8470.0,127.0,57411.0,-4289.0,-86820.0,-1926.0,72358.0,72358.0,NaN,14081.0,14081.0,NaN,880.0,16091.0,NaN,-71478.0,NaN,-10435.0,NaN,57411.0,Annual
4,2019-09-30,USD,69391.0,3423.0,NaN,-7248.0,-3760.0,12547.0,10495.0,-3176.0,289.0,55256.0,45896.0,-90976.0,-11954.0,66897.0,66897.0,NaN,14119.0,14119.0,NaN,781.0,6963.0,NaN,-66116.0,NaN,24311.0,NaN,55256.0,Annual
5,2018-09-30,USD,77434.0,3022.0,NaN,47665.0,12927.0,10903.0,13313.0,13332.0,-828.0,59531.0,16066.0,-87876.0,-74.0,72738.0,72738.0,NaN,13712.0,13712.0,NaN,669.0,6969.0,NaN,-72069.0,NaN,5624.0,NaN,59531.0,Annual
6,2017-09-30,USD,64225.0,2092.0,NaN,8838.0,14388.0,10157.0,12795.0,6347.0,2723.0,48351.0,-46446.0,-17974.0,7704.0,32900.0,32900.0,NaN,12769.0,12563.0,NaN,555.0,28662.0,NaN,-32345.0,NaN,-195.0,NaN,48351.0,Annual
7,2016-09-30,USD,66231.0,1316.0,NaN,-1867.0,-2351.0,10505.0,13548.0,-1044.0,-217.0,45687.0,-45977.0,-20890.0,-794.0,29722.0,29722.0,NaN,12150.0,11965.0,NaN,495.0,24954.0,NaN,-29227.0,NaN,-636.0,NaN,45687.0,Annual
8,2015-09-30,USD,81266.0,514.0,NaN,15188.0,-194.0,9200.0,11488.0,-611.0,238.0,53394.0,-56274.0,-17716.0,4382.0,35253.0,35253.0,NaN,11561.0,11561.0,NaN,543.0,27114.0,NaN,-34710.0,NaN,7276.0,NaN,53394.0,Annual
9,2014-09-30,USD,109133.0,339.0,NaN,13408.0,6361.0,1100.0,9571.0,6452.0,76.0,39510.0,-22579.0,-37549.0,6306.0,45000.0,45000.0,NaN,11031.0,11126.0,NaN,730.0,11960.0,NaN,-44270.0,NaN,-415.0,NaN,39510.0,Annual


In [85]:

# Laden der Kursdaten und erzeugen der durchschnittlichen daten für Annual und Quaterly
# 




# zusammenfügen von dataframes mit quaterly Daten
df_trimmed_qu = [df.iloc[:4] for df in dfs_qu]


# Krus für annual Data
min_date_an = all_reports_an['fiscalDateEnding'].min()
min_date_qu = all_reports_qu['fiscalDateEnding'].min()
max_date = pd.Timestamp.today().strftime('%Y-%m-%d')

data_an = yf.download(ticker,start=min_date_an, end=max_date)
data_an['year'] = data_an.index.year

all_reports_an['fiscalDateEnding'] = pd.to_datetime(all_reports_an['fiscalDateEnding'])
all_reports_an['year'] = all_reports_an['fiscalDateEnding'].dt.year
average_close_by_year = data_an.groupby('year')['Close'].mean().reset_index()
average_close_by_year.columns = ['year', 'avg_Kurs']
average_close_by_year['avg_Kurs'] = round(average_close_by_year['avg_Kurs'].astype(float),2)

all_reports_an = pd.merge(all_reports_an, average_close_by_year, how='inner',on='year')


#Kurs der TTM
data_qu = yf.download(ticker,start=min_date_qu, end=max_date)
average_close_by_ttm = data_qu.Close.mean()


# bilden ttm dataframe
ttm = all_reports_qu.select_dtypes(include=['number']).sum(axis=0)
ttm = pd.DataFrame(ttm).transpose()
ttm['year'] ='ttm'
ttm['avg_Kurs'] = average_close_by_ttm

#Verbinden der Dataframes
all_reports_an=all_reports_an.drop(columns=['fiscalDateEnding','reportedCurrency'])
balance_ttm = balance_qu[balance_qu['fiscalDateEnding'] == balance_qu['fiscalDateEnding'].max()].drop(columns = ['ReportType','fiscalDateEnding','reportedCurrency'])
ttm = pd.concat([ttm,balance_ttm], axis =1)

all_reports_an = pd.concat([all_reports_an,ttm]).reset_index(drop=True)

,fiscalDateEnding,reportedCurrency,operatingCashflow,paymentsForOperatingActivities,proceedsFromOperatingActivities,changeInOperatingLiabilities,changeInOperatingAssets,depreciationDepletionAndAmortization,capitalExpenditures,changeInReceivables,changeInInventory,profitLoss,cashflowFromInvestment,cashflowFromFinancing,proceedsFromRepaymentsOfShortTermDebt,paymentsForRepurchaseOfCommonStock,paymentsForRepurchaseOfEquity,paymentsForRepurchaseOfPreferredStock,dividendPayout,dividendPayoutCommonStock,dividendPayoutPreferredStock,proceedsFromIssuanceOfCommonStock,proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet,proceedsFromIssuanceOfPreferredStock,proceedsFromRepurchaseOfEquity,proceedsFromSaleOfTreasuryStock,changeInCashAndCashEquivalents,changeInExchangeRate,netIncome,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,investmentIncomeNet,netInterestIncome,interestIncome,interestExpense,nonInterestIncome,otherNonOperatingIncome,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,reportedEPS
0,2024-03-31,USD,22690.0,0.0,0.0,-11740.0,-5976.0,2836.0,1996.0,-8767.0,-190.0,23636.0,-310.0,-30433.0,4.0,23205.0,23205.0,0.0,3710.0,3710.0,0.0,0.0,0.0,0.0,-23205.0,0.0,0.0,0.0,23636.0,42271.0,90753.0,54950.0,48482.0,27900.0,6468.0,7903.0,14371.0,0.0,0.0,0.0,0.0,90753.0,0.0,0.0,2836.0,28058.0,4422.0,0.0,23636.0,24054.0,27900.0,30736.0,1.5300
1,2023-12-31,USD,39895.0,0.0,0.0,-8407.0,-9530.0,2848.0,2392.0,-11124.0,137.0,33916.0,1927.0,-30585.0,-7968.0,20139.0,20139.0,0.0,3825.0,3825.0,0.0,0.0,0.0,0.0,-20139.0,0.0,0.0,0.0,33916.0,54855.0,119575.0,71506.0,64720.0,40373.0,6786.0,7696.0,14482.0,0.0,0.0,0.0,1002.0,119575.0,0.0,0.0,2848.0,40323.0,6407.0,0.0,33916.0,35990.0,40373.0,43221.0,2.1800
2,2023-09-30,USD,21598.0,3113.0,0.0,14946.0,21006.0,2653.0,2163.0,21137.0,-952.0,22956.0,2394.0,-23153.0,3986.0,21003.0,21003.0,0.0,3758.0,3758.0,0.0,0.0,0.0,0.0,-21003.0,0.0,839.0,0.0,22956.0,40427.0,88496.0,55222.0,49071.0,26969.0,6151.0,7307.0,13458.0,984.0,-1002.0,18.0,1002.0,89498.0,47.0,0.0,2653.0,26998.0,4042.0,1002.0,22956.0,23305.0,28000.0,29622.0,1.4600
3,2023-06-30,USD,26380.0,0.0,0.0,5203.0,4454.0,3052.0,2093.0,3661.0,22.0,19881.0,437.0,-24048.0,0.0,17478.0,0.0,0.0,3849.0,3849.0,0.0,0.0,0.0,0.0,0.0,0.0,2769.0,0.0,19881.0,36413.0,80799.0,51357.0,45384.0,22998.0,5973.0,7442.0,13415.0,980.0,-998.0,18.0,998.0,81797.0,-247.0,0.0,3052.0,22733.0,2852.0,998.0,19881.0,19826.0,23731.0,26050.0,1.2600
4,2023-03-31,USD,28560.0,0.0,0.0,-16821.0,-17052.0,2898.0,2916.0,-17786.0,741.0,24160.0,2319.0,-25724.0,508.0,19594.0,19594.0,0.0,3650.0,3650.0,0.0,0.0,0.0,0.0,-19594.0,0.0,5155.0,0.0,24160.0,41976.0,94836.0,59061.0,52860.0,28318.0,6201.0,7457.0,13658.0,0.0,-12.0,12.0,930.0,94836.0,76.0,0.0,2898.0,28382.0,4222.0,0.0,24160.0,25326.0,29312.0,31216.0,1.5200
5,2022-12-31,USD,34005.0,703.0,0.0,-2186.0,-689.0,2916.0,3787.0,-6595.0,1807.0,29998.0,-1445.0,-35563.0,-16428.0,19475.0,19475.0,0.0,3768.0,3768.0,0.0,0.0,0.0,0.0,-19475.0,0.0,-3003.0,0.0,29998.0,50332.0,116151.0,73429.0,66822.0,36016.0,6607.0,7709.0,14316.0,868.0,-1003.0,135.0,1003.0,117154.0,-258.0,0.0,2916.0,35623.0,5625.0,1003.0,29998.0,28195.0,36626.0,38932.0,1.8800
6,2022-09-30,USD,24127.0,2755.0,0.0,21420.0,21468.0,2865.0,3289.0,18693.0,-435.0,20721.0,-1217.0,-26794.0,-2030.0,24428.0,24428.0,0.0,3703.0,3703.0,0.0,0.0,5465.0,0.0,-24428.0,0.0,-3884.0,0.0,20721.0,38095.0,89319.0,58491.0,52051.0,24894.0,6440.0,6761.0,13201.0,753.0,-827.0,74.0,827.0,90146.0,-163.0,0.0,2865.0,24657.0,3936.0,827.0,20721.0,18909.0,25484.0,27759.0,1.2900
7,2022-06-30,USD,22892.0,504.0,0.0,-6627.0,-3402.0,2805.0,2102.0,-3165.0,16.0,19442.0,4234.0,-27445.0,7942.0,21865.0,21865.0,0.0,3811.0,3811.0,0.0,0.0,0.0,0.0,-21865.0,0.0,19442.0,0.0,19442.0,35885.0,82240.0,53086.0,47074.0,23076.0,6012.0,6797.0,12809.0,

In [4]:
df = pd.read_csv('overview.csv')
df

,Symbol,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,Industry,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
0,AAPL,Common Stock,Apple Inc,Apple Inc. is an American multinational techno...,320193,NASDAQ,USD,USA,TECHNOLOGY,ELECTRONIC COMPUTERS,...,7.63,21.89,1.264,199.1,163.86,172.89,180.85,15334100000,2024-05-16,2024-05-10


In [7]:
overview

,Symbol,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,Industry,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
0,AAPL,Common Stock,Apple Inc,Apple Inc. is an American multinational techno...,320193,NASDAQ,USD,USA,TECHNOLOGY,ELECTRONIC COMPUTERS,...,7.63,21.89,1.264,199.1,163.86,172.89,180.85,15334100000,2024-05-16,2024-05-10


In [45]:
import requests
ticker = 'AAPL'
API_KEY = 'R8RUI8U2LWPPPXOG'

url_overview = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={ticker}&apikey={API_KEY}'
url_balance_sheet = f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={ticker}&apikey={API_KEY}'
url_cashflow= f'https://www.alphavantage.co/query?function=CASH_FLOW&symbol={ticker}&apikey={API_KEY}'
url_income_statement = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={ticker}&apikey={API_KEY}'

data_overview = requests.get(url_overview, verify=False).json()
data_balance_sheet = requests.get(url_balance_sheet, verify=False).json()
data_cashflow = requests.get(url_cashflow, verify=False).json()
data_income_statement = requests.get(url_income_statement, verify=False).json()

C:\Users\xv20361\.conda\envs\valueinv\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.alphavantage.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\xv20361\.conda\envs\valueinv\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.alphavantage.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\xv20361\.conda\envs\valueinv\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.alphavantage.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\User

In [46]:
data_cashflow

{'symbol': 'AAPL',
 'annualReports': [{'fiscalDateEnding': '2023-09-30',
   'reportedCurrency': 'USD',
   'operatingCashflow': '110543000000',
   'paymentsForOperatingActivities': '5703000000',
   'proceedsFromOperatingActivities': 'None',
   'changeInOperatingLiabilities': '1142000000',
   'changeInOperatingAssets': '7719000000',
   'depreciationDepletionAndAmortization': '11519000000',
   'capitalExpenditures': '10959000000',
   'changeInReceivables': '417000000',
   'changeInInventory': '1618000000',
   'profitLoss': '96995000000',
   'cashflowFromInvestment': '3705000000',
   'cashflowFromFinancing': '-108488000000',
   'proceedsFromRepaymentsOfShortTermDebt': '-7956000000',
   'paymentsForRepurchaseOfCommonStock': '77550000000',
   'paymentsForRepurchaseOfEquity': '77550000000',
   'paymentsForRepurchaseOfPreferredStock': 'None',
   'dividendPayout': '15025000000',
   'dividendPayoutCommonStock': '15025000000',
   'dividendPayoutPreferredStock': 'None',
   'proceedsFromIssuanceOfC

In [49]:
cashflow_qu.loc[cashflow_qu['fiscalDateEnding']=='2023-03-31']

,fiscalDateEnding,reportedCurrency,operatingCashflow,paymentsForOperatingActivities,proceedsFromOperatingActivities,changeInOperatingLiabilities,changeInOperatingAssets,depreciationDepletionAndAmortization,capitalExpenditures,changeInReceivables,changeInInventory,profitLoss,cashflowFromInvestment,cashflowFromFinancing,proceedsFromRepaymentsOfShortTermDebt,paymentsForRepurchaseOfCommonStock,paymentsForRepurchaseOfEquity,paymentsForRepurchaseOfPreferredStock,dividendPayout,dividendPayoutCommonStock,dividendPayoutPreferredStock,proceedsFromIssuanceOfCommonStock,proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet,proceedsFromIssuanceOfPreferredStock,proceedsFromRepurchaseOfEquity,proceedsFromSaleOfTreasuryStock,changeInCashAndCashEquivalents,changeInExchangeRate,netIncome,ReportType
4,2023-03-31,USD,28560.0,NaN,NaN,-16821.0,-17052.0,2898.0,2916.0,-17786.0,741.0,24160.0,2319.0,-25724.0,508.0,19594.0,19594.0,NaN,3650.0,3650.0,NaN,NaN,NaN,NaN,-19594.0,NaN,5155.0,NaN,24160.0,Quarterly
5,2023-03-31,USD,28560.0,NaN,NaN,NaN,NaN,2898.0,2916.0,17786.0,-741.0,24160.0,2319.0,-25724.0,0.0,NaN,0.0,NaN,3650.0,3650.0,NaN,0.0,0.0,NaN,0.0,NaN,5155.0,NaN,24160.0,Quarterly


In [104]:
data_qu

,Open,High,Low,Close,Adj Close,Volume
0,191.630005,194.479996,191.259995,193.970001,192.945312,85069600
1,193.779999,193.880005,191.759995,192.460007,191.443314,31458200
2,191.570007,192.979996,190.619995,191.330002,190.319275,46920300
3,189.839996,192.020004,189.199997,191.809998,190.796722,45094300
4,191.410004,192.669998,190.240005,190.679993,189.672684,46778000
5,189.259995,189.990005,187.039993,188.610001,187.613632,59922200
6,189.160004,189.300003,186.600006,188.080002,187.086426,46638100
7,189.679993,191.699997,188.470001,189.770004,188.767502,60750200
8,190.500000,191.190002,189.779999,190.539993,189.533432,41342300
9,190.229996,191.179993,189.630005,190.690002,189.682648,41573900


In [1]:
import pandas as pd
import streamlit as st

# example multi-index dataframe
data = {'level_0': [1, 2, 3],
        'level_1': ['A', 'B', 'C'],
        '01/23/2023': ['20%', '30%', '40%'],
        '01/24/2023': ['10%', '20%', '30%'],
        '01/25/2023': ['15%', '25%', '35%'],
        '01/26/2023': ['5%', '10%', '15%'],
        '01/27/2023': ['1%', '2%', '3%']}

df = pd.DataFrame(data)
df = df.set_index(['level_0', 'level_1'])

def make_bar_style(x):
    if '%' in str(x):
        x = float(x.strip('%'))
        return f"background: linear-gradient(90deg,#5fba7d {x}%, transparent {x}%); width: 10em"
    return ''


st.dataframe(df.style.applymap(make_bar_style))

C:\Users\xv20361\AppData\Local\Temp\ipykernel_26652\1754344802.py:23: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  st.dataframe(df.style.applymap(make_bar_style))
2024-05-17 15:09:06.047 
  command:

    streamlit run C:\Users\xv20361\.conda\envs\valueinv\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()